## GLUE dataset과 Huggingface

### GLUE Benchmark Dataset

Pretrained model의 성능을 측정하기 위해 최근은 SQuAD 등 기존에 유명한 데이터셋 한 가지만 가지고 성능을 논하는 것이 아니라, classification, summarization, reasoning, Q&A 등 NLP 모델의 성능을 평가할 수 있는 다양한 task를 해당 모델 하나만을 이용해 모두 수행해 보면서 종합적인 성능을 논하는 것이 일반화되었습니다.  
  
그중 NLP 모델의 성능을 측정하기 위한 데이터셋으로 최근 활용되는 대표적인 것 중에 General Language Understanding Evaluation(GLUE) benchmark Dataset이 있습니다. 총 10가지 데이터셋이 있습니다. 각각의 개요는 다음과 같습니다.  
  
* CoLA : 문법에 맞는 문장인지 판단
* MNLI : 두 문장의 관계 판단(entailment, contradiction, neutral)
* MNLI-MM : 두 문장이 안 맞는지 판단
* MRPC : 두 문장의 유사도 평가
* SST-2 : 감정분석
* STS-B : 두 문장의 유사도 평가
* QQP : 두 질문의 유사도 평가
* QNLI : 질문과 paragraph 내 한 문장이 함의 관계(entailment)인지 판단
* RTE : 두 문장의 관계 판단(entailment, not_entailment)
* WNLI : 원문장과 대명사로 치환한 문장 사이의 함의 관계 판단  
  
GLUE 홈페이지에는 위 10가지 task에 대한 상세한 설명, 그리고 Leaderboard를 운영하고 있습니다. 한 가지 task에만 최적화된 모델이 아니라, 다양한 형태의 문제를 골고루 잘 푸는 모델을 찾기 위한 노력이 계속되고 있습니다.

### Huggingface가 제공하는 GLUE task 예제 코드
Huggingface와 같은 NLP framework는 해당 framework를 활용하여 새로 만들어진 모델의 성능을 빠르게 평가해 볼 수 있도록 하는 예제 코드를 제공하고 있습니다. 이런 예제 코드가 없다면 모델을 새로 만들 때마다 그 성능을 비교 측정해 보기 위한 작업이 너무나 번거롭겠죠? 오늘은 우선 Huggingface의 예제 코드를 들여다보는 것으로부터 시작해 보겠습니다.  
우선 소스코드 프로젝트를 통해 다시 설치해 봅시다. 설치하지 않으면 오류가 발생하므로 꼭 설치해 주세요.

In [1]:
# $ cd ~/aiffel && git clone https://github.com/huggingface/transformers.git
# $ cd transformers && pip install -e .
# $ pip install datasets

# $ cd ~/aiffel/transformers
# $ python examples/tensorflow/text-classification/run_glue.py \
# 	--model_name_or_path bert-base-cased \
# 	--task_name mrpc \
# 	--output_dir ./models/mrpc \
# 	--overwrite_output_dir \
# 	--do_train \
# 	--do_eval \
# 	--num_train_epochs 1 \
# 	--save_steps 20000

위 코드는 10가지 GLUE task 중 'mrpc' task를 수행하는 예제 코드입니다. 이 코드는 Huggingface의 framework 기반으로 BERT `bert-base-cased`을 활용하여 'mrpc' task를 수행합니다. 만약 `task_name` 및 다른 파라미터를 적절히 변경한 후 수행하면 다른 GLUE task도 간단히 수행해 볼 수 있을 것입니다. model도 다양하게 바꾸어 보면서 수월하게 수행 가능할 것입니다. 이 예제만으로도 NLP framework의 강력함을 손쉽게 느껴볼 수 있을 것입니다.

## 커스텀 프로젝트 제작 (1) Processor

### mrpc 데이터셋 분석
본격적으로 Huggingface framework를 활용해 봅시다. 언제나 그렇듯, 프로젝트를 수행하기 위한 첫 단계는 데이터를 분석하는 것입니다.

In [3]:
import os
import numpy as np
from argparse import ArgumentParser
import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import BertTokenizer, TFBertForSequenceClassification, AutoConfig
from dataclasses import asdict
from transformers.data.processors.utils import DataProcessor, InputExample, InputFeatures

GLUE 데이터셋은 홈페이지에서 원본을 다운로드할 수도 있지만, 이번에는 tensorflow_datasets에서 제공하는 것을 이용해 보겠습니다.

In [4]:
data, info = tfds.load('glue/mrpc', with_info=True)
info.splits['train'].num_examples

INFO:absl:Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: glue/mrpc/1.0.0
INFO:absl:Load dataset info from /tmp/tmpgotiqpcjtfds
INFO:absl:Field info.description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_name from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.citation from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.location from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.splits from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.module_name from disk and from code do not match. Keeping the one from code.
INFO:absl:Generating dataset glue (/aiffel/tensorflow_datasets/glue/mrpc/1.0.0)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

INFO:absl:Downloading https://firebasestorage.googleapis.com/v0/b/mtl-sentence-representations.appspot.com/o/data%2Fmrpc_dev_ids.tsv?alt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc into /aiffel/tensorflow_datasets/downloads/fire.goog.com_v0_b_mtl-sent-repr.apps.com_o_2FjSIMlCiqs1QSmIykr4IRPnEHjPuGwAz5i40v8K9U0Z8.tsvalt=media&token=ec5c0836-31d5-48f4-b431-7480817f1adc.tmp.2f88045dc1284d49b82dd1684e9bb31a...
INFO:absl:Downloading https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_test.txt into /aiffel/tensorflow_datasets/downloads/dl.fbaip.com_sente_sente_msr_parap_test0PdekMcyqYR-w4Rx_d7OTryq0J3RlYRn4rAMajy9Mak.txt.tmp.2bbd9f63e6cd4afebb3b505d0a6bf55e...
INFO:absl:Downloading https://dl.fbaipublicfiles.com/senteval/senteval_data/msr_paraphrase_train.txt into /aiffel/tensorflow_datasets/downloads/dl.fbaip.com_sente_sente_msr_parap_trainfGxPZuQWGBti4Tbd1YNOwQr-OqxPejJ7gcp0Al6mlSk.txt.tmp.50371afa3866421988d59f2e30d9f34d...


Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

...

Generating test examples...:   0%|          | 0/1725 [00:00<?, ? examples/s]

Shuffling glue-test.tfrecord...:   0%|          | 0/1725 [00:00<?, ? examples/s]

INFO:absl:Done writing glue-test.tfrecord. Number of examples: 1725 (shards: [1725])
INFO:absl:Constructing tf.data.Dataset for split None, from /aiffel/tensorflow_datasets/glue/mrpc/1.0.0


Dataset glue downloaded and prepared to /aiffel/tensorflow_datasets/glue/mrpc/1.0.0. Subsequent calls will reuse this data.


3668

`data`는 `tf.data.Dataset`을 상속받은 클래스의 형태일 것입니다. 우선 1개의 데이터만 가져다가 어떻게 생겼는지 확인해 봅시다.

In [7]:
data['train'].take(1)

<TakeDataset shapes: {idx: (), label: (), sentence1: (), sentence2: ()}, types: {idx: tf.int32, label: tf.int64, sentence1: tf.string, sentence2: tf.string}>

데이터셋 안에 어떤 항목이 정의되어 있는지 확인할 수 있었습니다. 실제 내용도 한번 확인해 볼까요?

In [8]:
examples = data['train'].take(1)
for example in examples:
    sentence1 = example['sentence1']
    sentence2 = example['sentence2']
    label = example['label']
    print(sentence1)
    print(sentence2)
    print(label)

tf.Tensor(b'The identical rovers will act as robotic geologists , searching for evidence of past water .', shape=(), dtype=string)
tf.Tensor(b'The rovers act as robotic geologists , moving on six wheels .', shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)


### Processor의 활용
우리는 지난 시간에, `Huggingface transformers`에서 task별로 데이터셋을 가공하는 일반적인 클래스 구조인 `Processor`에 대해 다룬 바 있습니다.  
  
아래는 추상클래스인 `Processor`를 한번 상속받은 후, Sequence Classification task를 수행하는 모델의 `Processor` 추상클래스인 `DataProcessor`mm입니다.

## 커스텀 프로젝트 제작 (2) Tokenizer와 Model

## 커스텀 프로젝트 제작 (3) Train/Evaluation과 Test